In [74]:
!pip install conll-df

  Running setup.py bdist_wheel for conll-df ... error
  Complete output from command /home/stanislaw/anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-z995cruc/conll-df/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/pip-wheel-yhoxhezt --python-tag cp36:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib
  creating build/lib/conll_df
  copying conll_df/__init__.py -> build/lib/conll_df
  copying conll_df/conll_df.py -> build/lib/conll_df
  installing to build/bdist.linux-x86_64/wheel
  running install
  running install_lib
  creating build/bdist.linux-x86_64
  creating build/bdist.linux-x86_64/wheel
  creating build/bdist.linux-x86_64/wheel/conll_df
  copying build/lib/conll_df/__init__.py -> build/bdist.linux-x86_64/wheel/conll_df
  copying build/lib/conll_df/conll_df.py -> build/bdist.linux-x86_64/w

In [10]:
import spacy
from tqdm import tqdm

In [2]:
import pandas as pd
from conll_df import conll_df

In [3]:
nlp=spacy.load('../models/pos_NKJP_word2vec/model-final/')

In [8]:
path='../data/dependency_trees/UD_Polish-LFG-master/pl_lfg-ud-train.conllu'

In [88]:
df=conll_df(path, skip_meta=True)

In [91]:
df=df[['w','l']]

In [119]:
for i,g in df.groupby(level=1):
    break

In [128]:
g

w              l
file                   s i                               
pl_lfg-ud-train.conllu 1 1  100-tysięcznym  100-tysięczny
                         2     Grudziądzem      Grudziądz
                         3          rządzi        rządzić
                         4          lewica         lewica
                         5               .              .

In [132]:
orth_tokens=list(g.w)
lemma_tokens=list(g.l)

sent=' '.join(orth_tokens)
sent

'100-tysięcznym Grudziądzem rządzi lewica .'

In [127]:
sent

'100-tysięcznym Grudziądzem rządzi lewica .'

In [129]:
analysed_sent=nlp(sent)

In [137]:
for token, orth, lemma in zip(analysed_sent, orth_tokens, lemma_tokens):
    print(lemma, token.lemma_)

100-tysięczny 100-tysięcznym
Grudziądz Grudziądzem
rządzić rządzi
lewica lewica
. .


In [140]:
len(analysed_sent)

5

In [141]:
len(lemma_tokens)

5

In [34]:
def count_matching_lemmas(analysed_sentence, true_lemmas):
    if len(analysed_sentence) != len(true_lemmas):
        raise Exception(f'{len(analysed_sentence)} not equals {len(true_lemmas)}')
    score=0
    for token, lemma in zip(analysed_sentence, true_lemmas):
        if(token.lemma_==lemma):
            score+=1
            
    return score

In [38]:
def load_lemmas_df(path):
    df=conll_df(path, skip_meta=True)
    df=df[['w','l']]
    return df

In [41]:
mismatch_indexes=[]

def evaluate_lemmas(data_path, nlp):
    word_count=0
    match_count=0

    df = load_lemmas_df(data_path)
    for index, sentence_group in tqdm(df.groupby(level=1)):
        orth_tokens = list(sentence_group.w)
        lemma_tokens = list(sentence_group.l)
        sent = ' '.join(orth_tokens)
        analysed_sent = nlp(sent)
        try:
            current_match = count_matching_lemmas(analysed_sent, lemma_tokens)
            word_count += len(orth_tokens)
            match_count += current_match
        except:
            mismatch_indexes.append(index)
        
    return match_count / word_count

In [42]:
evaluate_lemmas(path, nlp)

100%|██████████| 13774/13774 [01:23<00:00, 165.30it/s]


0.5023166578732793